In [28]:
import os
from os.path import join, exists

In [48]:

# Assumes a folder structure with /base_path/mouse/session:
# base_path
# ├── gmou82
# │   ├── 20221010_gmou83
# │   │   ├── 20221010_gmou83.bottom.depth.avi
# │   │   ├── 20221010_gmou83.bottom.device_timestamps.npy
# │   │   ├── 20221010_gmou83.bottom.ir.avi
# │   │   ├── 20221010_gmou83.bottom.system_timestamps.npy
# │   │   ├── 20221010_gmou83.matched_timestamps.npy
# │   │   ├── 20221010_gmou83.top.depth.avi
# │   │   ├── 20221010_gmou83.top.device_timestamps.npy
# │   │   ├── 20221010_gmou83.top.ir.avi
# │   │   ├── 20221010_gmou83.top.system_timestamps.npy
# │   │   └── 20221010_gmou83.txt
#     └── 20221011_gmou83
# │       ├── 20221011_gmou83.bottom.depth.avi
# │       ├── 20221011_gmou83.bottom.device_timestamps.npy
# │       ├── 20221011_gmou83.bottom.ir.avi
# │       ├── 20221011_gmou83.bottom.system_timestamps.npy
# │       ├── 20221011_gmou83.matched_timestamps.npy
# │       ├── 20221011_gmou83.top.depth.avi
# │       ├── 20221011_gmou83.top.device_timestamps.npy
# │       ├── 20221011_gmou83.top.ir.avi
# │       ├── 20221011_gmou83.top.system_timestamps.npy
# │       └── 20221011_gmou83.txt


In [42]:
# top bottom params
path_to_script = '~/datta-lab/top-bottom-moseq/scripts/process_session.py'
path_to_config = '/n/groups/datta/Jonah/Thermistor_only_recordings/top_bottom_dataset2/top_bottom_config.yaml'
sync_command = f'python {path_to_script} {{prefix}} {path_to_config}'


# data to use
base_path = '/n/groups/datta/Jonah/Thermistor_only_recordings/'
mice_to_use = ['gmou83']
dates_by_mice = {'gmou83': ['20221010', '20221011']}  # or set to None to use all sessions per mouse

# sbatch params
use_sbatch = True  # if false, just output python commands
job_time = '8:00:00'
mem = '64GB'
conda_env = 'top_bottom_moseq_37'
sbatch_prefix = f'sbatch -c 1 --mem={mem} -p gpu --gres=gpu:1 -t {job_time} -o {{srun_out_path}} --wrap'
wrap_prefix = f'eval "$(conda shell.bash hook)"; conda activate {conda_env}; module load gcc/6.2.0 ffmpeg;'
outfile_path = './batch_scripts'  # relative to base_path
outfile = 'batch_topbottom_gmou83.sh'  


In [43]:
# find all sessions to sync.
raw_data_folders = []
for mouse in mice_to_use:
    data_path = os.path.join(base_path, mouse)
    dir_contents = os.listdir(data_path)
    abs_path_dirs = [os.path.join(data_path,f) for f in dir_contents]
    if dates_by_mice:
        abs_path_dirs = [f for f in abs_path_dirs if any([date in f for date in dates_by_mice[mouse]])]
    data_folders = [f for f in abs_path_dirs if (os.path.isdir(f))]
    raw_data_folders.extend(data_folders)
raw_data_folders

['/n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221011_gmou83',
 '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221010_gmou83']

In [44]:
if not exists(join(base_path, outfile_path)):
    os.makedirs(join(base_path, outfile_path))

In [45]:
# make the batch script
batch_file = join(base_path, outfile_path, outfile)
with open(batch_file, 'w') as f:
    for i,folder in enumerate(raw_data_folders):
        prefix = join(folder, os.path.split(folder)[1])
        this_sync_command = sync_command.format(prefix=prefix)
        if use_sbatch:
            full_line = f'{sbatch_prefix.format(srun_out_path=join(folder, "%j.out"))} \'{wrap_prefix} {this_sync_command};\'\n'
        else:
            full_line = f'{this_sync_command};\n'
        print(full_line)
        f.write(full_line)

print(f'Saved batch script to {batch_file}')

sbatch -c 1 --mem=64GB -p gpu --gres=gpu:1 -t 8:00:00 -o /n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221011_gmou83 --wrap 'eval "$(conda shell.bash hook)"; conda activate top_bottom_moseq_37; module load gcc/6.2.0 ffmpeg; python ~/datta-lab/top-bottom-moseq/scripts/process_session.py /n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221011_gmou83/20221011_gmou83 /n/groups/datta/Jonah/Thermistor_only_recordings/top_bottom_dataset2/top_bottom_config.yaml;'

sbatch -c 1 --mem=64GB -p gpu --gres=gpu:1 -t 8:00:00 -o /n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221010_gmou83 --wrap 'eval "$(conda shell.bash hook)"; conda activate top_bottom_moseq_37; module load gcc/6.2.0 ffmpeg; python ~/datta-lab/top-bottom-moseq/scripts/process_session.py /n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221010_gmou83/20221010_gmou83 /n/groups/datta/Jonah/Thermistor_only_recordings/top_bottom_dataset2/top_bottom_config.yaml;'

Saved batch script to /n/groups/datt

In [ ]:
# spacer

In [ ]:
# spacer

In [46]:
# make executable
os.system(f'chmod u+x {batch_file}')

0

In [47]:
# run the script
os.system(f'{batch_file}')

Submitted batch job 63449788
Submitted batch job 63449789


0